In [8]:
import numpy as np
from sklearn.preprocessing import normalize
import math


**Non Interactive DP Data Release**

Data Pre Processing

In [ ]:
def pre_processing(dataset,epsilon_mu):
    '''Input Numpy Matrix of dimension (m,n) and privacy param epsion '''
    pre_normalized_dataset = normalize(dataset,axis=0)
    #deriving dp mean
    avg = np.mean(pre_normalized_dataset,axis=1)
    m,n = pre_normalized_dataset.shape
    loc_param = 2*math.sqrt(m)/(n*epsilon_mu)
    laplace_noise = np.random.laplace(0,loc=loc_param,(m,1))
    mu_dp = avg+laplace_noise
    #centerize the data
    mat_1 = np.ones((1,n))
    centralized_dataset = dataset - np.matmul(mu_dp,mat_1)
    final_dataset = np.mean(centralized_dataset,axis=0)
    return final_dataset,mu_dp


RON_Projection

In [ ]:
def ron_projection(pre_processed_dataset,p):
    m,n = pre_processed_dataset.shape
    A = np.random.uniform(0,1,(m,m))
    Q,R = numpy.linalg.qr(a, mode='complete')
    #constructing a Ron Projection Matrix
    W = Q[:,:p]
    projected_data = np.matmul(np.transpose(W),pre_processed_dataset)
    return projected_data,W


RON Gauss For Unsupervised Learning

In [ ]:
def ron_gauss_for_unsupervised(dataset,dimension_p,epsilon_mu,epsilon_sigma):
    preprocessed_data = pre_processing(dataset,epsilon_mu)
    ron_projected_data = ron_projection(preprocessed_data,dimension_p)
    m,n = dataset.shape
    loc_param = 2*math.sqrt(dimension_p)/(n*epsilon_sigma)
    laplace_noise = np.random.laplace(0,loc=loc_param,(dimension_p,dimension_p))
    dp_cov = (1/n)*np.matmul(ron_projected_data,np.transpose(ron_projected_data))+laplace_noise
    synthesized_dp_data = np.random.multivariate_normal(0, dp_cov, size=(dimension_p,n))
    return synthesized_dp_data

RON Gauss For Supervised Learning

In [ ]:
def ron_gauss_for_supervised(dataset,training_label,a,dimension_p,epsilon_mu,epsilon_sigma):
    #label.shape = (n,1)
    preprocessed_data = pre_processing(dataset,epsilon_mu)
    ron_projected_data = ron_projection(preprocessed_data,dimension_p)
    p,n = ron_projected_data.shape
    augmented_ron = np.hstack((ron_projected_data,np.transpose(training_label)))
    sqp = math.sqrt(p)
    loc_param = (2*sqp+4*a*sqp+a*a)/(n*epsilon_sigma)
    laplace_noise = np.random.laplace(0,loc=loc_param,(dimension_p+1,dimension_p+1))
    dp_cov = (1/n)*np.matmul(augmented_ron,np.transpose(augmented_ron))+laplace_noise
    synthesized_dp_data = np.random.multivariate_normal(0, dp_cov, size=(dimension_p+1,n))
    return synthesized_dp_data
